In [ ]:
# default_exp design

# Design
> All design-related functions

In [ ]:
# exporti
from typing import NamedTuple

import jax.numpy as jnp
import matplotlib.pyplot as plt
from fastcore.basics import patch_to
from matplotlib.colors import ListedColormap
from typing import NamedTuple

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from fastcore.basics import patch_to
from inverse_design.utils import batch_conv2d, conv2d, dilute
from matplotlib.colors import ListedColormap
from types import SimpleNamespace
from inverse_design.utils import and_, not_

In [ ]:
# export
UNASSIGNED = 0
VOID = 1
SOLID = 2
PIXEL_IMPOSSIBLE = 3
PIXEL_EXISTING = 4
PIXEL_POSSIBLE = 5
PIXEL_REQUIRED = 6
TOUCH_REQUIRED = 7
TOUCH_INVALID = 8
TOUCH_EXISTING = 9
TOUCH_VALID = 10
TOUCH_FREE = 11
TOUCH_RESOLVING = 12

In [ ]:
# export

class Design(NamedTuple):
    void: jnp.ndarray
    solid: jnp.ndarray
    void_pixel_impossible: jnp.ndarray
    void_pixel_existing: jnp.ndarray
    void_pixel_possible: jnp.ndarray
    void_pixel_required: jnp.ndarray
    solid_pixel_impossible: jnp.ndarray
    solid_pixel_existing: jnp.ndarray
    solid_pixel_possible: jnp.ndarray
    solid_pixel_required: jnp.ndarray
    void_touch_existing: jnp.ndarray
    void_touch_valid: jnp.ndarray
    void_touch_invalid: jnp.ndarray
    void_touch_free: jnp.ndarray
    void_touch_resolving: jnp.ndarray
    solid_touch_invalid: jnp.ndarray
    solid_touch_existing: jnp.ndarray
    solid_touch_valid: jnp.ndarray
    solid_touch_invalid: jnp.ndarray
    solid_touch_free: jnp.ndarray
    solid_touch_resolving: jnp.ndarray
    
    @property
    def shape(self):
        return self.unassigned.shape
    
    @property
    def unassigned(self):
        return and_(not_(self.void), not_(self.solid))
    
    def copy(self, **kwargs):
        kwargs = {name: kwargs.get(name, getattr(self, name)) for name in self._fields}
        return Design(*kwargs.values())

In [ ]:
# export
def new_design(shape):
    return Design(
        void=jnp.zeros(shape, dtype=jnp.float32),
        solid=jnp.zeros(shape, dtype=jnp.float32),
        void_pixel_impossible=jnp.zeros(shape, dtype=jnp.float32),
        void_pixel_existing=jnp.zeros(shape, dtype=jnp.float32),
        void_pixel_possible=jnp.ones(shape, dtype=jnp.float32),
        void_pixel_required=jnp.zeros(shape, dtype=jnp.float32),
        solid_pixel_impossible=jnp.zeros(shape, dtype=jnp.float32),
        solid_pixel_existing=jnp.zeros(shape, dtype=jnp.float32),
        solid_pixel_possible=jnp.ones(shape, dtype=jnp.float32),
        solid_pixel_required=jnp.zeros(shape, dtype=jnp.float32),
        void_touch_invalid=jnp.zeros(shape, dtype=jnp.float32),
        void_touch_existing=jnp.zeros(shape, dtype=jnp.float32),
        void_touch_valid=jnp.ones(shape, dtype=jnp.float32),
        void_touch_free=jnp.zeros(shape, dtype=jnp.float32),
        void_touch_resolving=jnp.zeros(shape, dtype=jnp.float32),
        solid_touch_invalid=jnp.zeros(shape, dtype=jnp.float32),
        solid_touch_existing=jnp.zeros(shape, dtype=jnp.float32),
        solid_touch_valid=jnp.ones(shape, dtype=jnp.float32),
        solid_touch_free=jnp.zeros(shape, dtype=jnp.float32),
        solid_touch_resolving=jnp.zeros(shape, dtype=jnp.float32),
    )

In [ ]:
# export
def design_mask(design, dtype=float):
    return jnp.where(design.void > 0.5, -1.0, 1.0)

In [ ]:
# export
def visualize(design):
    zero = jnp.zeros_like(design.unassigned, dtype=jnp.uint8)
    b = lambda float_mask: float_mask > 0.5
    design_vis = dict(
        design=zero.at[b(design.unassigned)].set(UNASSIGNED).at[b(design.void)].set(VOID).at[b(design.solid)].set(SOLID),
        void_pixels=zero.at[b(design.void_pixel_possible)].set(PIXEL_POSSIBLE).at[b(design.void_pixel_impossible)].set(PIXEL_IMPOSSIBLE).at[b(design.void_pixel_existing)].set(PIXEL_EXISTING).at[b(design.void_pixel_required)].set(PIXEL_REQUIRED),
        solid_pixels=zero.at[b(design.solid_pixel_possible)].set(PIXEL_POSSIBLE).at[b(design.solid_pixel_impossible)].set(PIXEL_IMPOSSIBLE).at[b(design.solid_pixel_existing)].set(PIXEL_EXISTING).at[b(design.solid_pixel_required)].set(PIXEL_REQUIRED),
        void_touches=zero.at[b(design.void_touch_valid)].set(TOUCH_VALID).at[b(design.void_touch_invalid)].set(TOUCH_INVALID).at[b(design.void_touch_existing)].set(TOUCH_EXISTING).at[b(design.void_touch_free)].set(TOUCH_FREE).at[b(design.void_touch_resolving)].set(TOUCH_RESOLVING),
        solid_touches=zero.at[b(design.solid_touch_valid)].set(TOUCH_VALID).at[b(design.solid_touch_invalid)].set(TOUCH_INVALID).at[b(design.solid_touch_existing)].set(TOUCH_EXISTING).at[b(design.solid_touch_free)].set(TOUCH_FREE).at[b(design.solid_touch_resolving)].set(TOUCH_RESOLVING),
    )
    nx, ny = design.shape
    _cmap = ListedColormap(colors={UNASSIGNED: "#929292", VOID: "#cbcbcb", SOLID: "#515151", PIXEL_IMPOSSIBLE: "#8dd3c7", PIXEL_EXISTING: "#ffffb3", PIXEL_POSSIBLE: "#bebada", PIXEL_REQUIRED: "#fb7f72", TOUCH_REQUIRED: "#00ff00", TOUCH_INVALID: "#7fb1d3", TOUCH_EXISTING: "#fdb462", TOUCH_VALID: "#b3de69", TOUCH_FREE: "#fccde5", TOUCH_RESOLVING: "#e0e0e0"}.values(), name="cmap")
    fig, axs = plt.subplots(1, 5, figsize=(15,3*nx/ny))
    for i, (title, arr) in enumerate(design_vis.items()):
        ax = axs[i]
        ax.set_title(title.replace("_", " "))
        ax.imshow(arr, cmap=_cmap, vmin=UNASSIGNED, vmax=TOUCH_RESOLVING)
        ax.set_yticks(jnp.arange(nx)+0.5, ["" for i in range(nx)])
        ax.set_xticks(jnp.arange(ny)+0.5, ["" for i in range(ny)])
        ax.set_yticks(jnp.arange(nx), [f"{i}" for i in range(nx)], minor=True)
        ax.set_xticks(jnp.arange(ny), [f"{i}" for i in range(ny)], minor=True)
        ax.set_xlim(-0.5, ny-0.5)
        ax.set_ylim(nx-0.5, -0.5)
        ax.grid(visible=True, which="major", c="k")

@patch_to(Design)
def _repr_html_(self):
    visualize(self)
    return ""

In [ ]:
new_design((6, 8))